Frequently Bought Toghether Items

Inizializzazione, parametri e altre cose che tutti devono sapere

In [ ]:
#import numpy as np
#import matplotlib.pyplot as plt
import random
import networkx as nx
import node2vec
from gensim.models import Word2Vec

#Nodo target
target = 3

#Grafo completo FBT
full_graph = nx.Graph()

#Grafo cluster selezionato
clustered_graph = nx.Graph()

#File di lettura
file_name = "Amazon0302.txt"
#file_name = "grafo_esempio.txt"

#Grafo directed se True o uniderected se False
directed = True

#embedding hyperparameters
#default = 1, 1, 10, 80
p=1
q=1
num_walks=10
walk_length=80

In [ ]:
### LOAD GRAPH ###
print("loading graph " + file_name)

if directed:
    ##genera il grafo directed utilizzando nx.DiGraph
    full_graph = nx.read_edgelist(file_name, nodetype=int, create_using=nx.DiGraph)
		
else:
    ##genera il grafo undirected utilizzando nx.Graph
    full_graph = nx.read_edgelist(file_name)
    
nx.set_edge_attributes(full_graph, 1, name='weight')
print(full_graph)

print("finished")

Embedding using stanford's node2vec

In [ ]:
G = node2vec.Graph(full_graph, directed, p, q)
G.preprocess_transition_probs()
walks = G.simulate_walks(num_walks, walk_length)

In [ ]:
walks = [list(map(str, walk)) for walk in walks]
model = Word2Vec(walks, window=10, min_count=0, sg=1, workers=8)
#model.save_word2vec_format("out.emb")

Clustering

In [ ]:
print(model)
vector = model.wv[1]
print(vector)

NUMBER_OF_ITERATIONS = 10

#k_means_cluster
class Point:
    def __init__(self, x, y):
        self.vector = x
        self.assigned = y

#calculate distance between two points
def distance(point1, point2):
    dist = 0
    for i in range(point1.vector.len()):
        dist += (point1.vector[i] - point2.vector[i])**2
    return dist

#calculate S score of point given array of centers and other points
def calculate_silhouette(point, centers, points):
    a_i = 0.0
    b_i = float('inf')
    cluster_size = 0
    members = 0

    for i in range(points.len()):
        if points[i].assigned == point.assigned:
            a_i += distance(point, points[i])
            members += 1

    if members < 2:
        return 0
    else:
        a_i/= (members-1)

    for k in range(centers.len()):
        if not(k == point.assigned):
            sum = 0.0
            number = 0
            for i in range(points.len()):
                if points[i].assigned == k:
                    sum += distance(point, points[i])
                    number +=1
            if not(number == 0):
                sum /= number
                if sum < b_i:
                    b_i = sum
    
    if centers.len() == 1:
        b_i = 0

    return (b_i - a_i)/max(a_i, b_i)

# calculate total silhouette score for clustering
def calculate_silhouette_score(points, centers):
    ss = 0.0
    for point in points:
        best_assign = -1
        min_dist = float('inf')
        for k in range(centers.len()):
            if distance(point, centers[k] < min_dist):
                point.assigned = k
                min_dist = ditance(point, centers[k])

    for p in points:
        ss += calculate_silhouette(p, centers, points)
        
    return ss/points.size()

def k_means_plus_plus(points, k_chosen):
    ##init with kmeans++##
    centroids = []
    distances = []
    probabilities = []

    random_first_point = randrange(points.len())
    centroids.append(points[random_first_point])
    sum = 0.0
    for p in points:
        current = distance(centroids[0], p)
        distances.append(current)
        sum += current**2
        probabilities.append(sum)
    #choose a point with weighted probability for kmeans ++
    for i in range(k_chosen):
        if not(i == 0):
            chosen = uniform(0.0, sum)
            c = 0
            for j in range(points.len()):
                if chosen >= probabilities[j]:
                    c = j

        centroids.append(points[c])
        sum = 0
        for j in range(points.len()):
            current = distance(centroids[i], points [j])
            if(distances[j] > current):
                distances[j] = current
                points[j].assigned = i
            sum += distances[j]**2
            probabilities[j] = sum

    ##start kmeans##
    for iter in range(NUMBER_OF_ITERATIONS):
        changed = False
        for k in range(k_chosen):
            med_vec = []
            for j in range(points[0].vector.len()):
                med = 0
                pts = 0
                for p in points:
                    if(p.assigned == k):
                        med += p.vector[j]
                        pts += 1
                med /= pts
                med_vec.append(med)
            centroids[k] = Point(med_vec, k)

        for p in points:
            best_dist = float('inf')
            best_k = 0
            for k in k_chosen:
                dist = distance(p, centroids[k])
                if(dist < best_dist):
                    best_dist = dist
                    best_k = k
            if not(best_k == p.assigned):
                changed = True
                p.assigned = best_k
                
        if not changed:
            break
        else
            changed = False

    #find closest point to centroid
    for k in range(k_chosen):
        dist = float('inf')
        i = -1
        for p in range(point.len()):
            dist_p = distance(points[p], centroids[k])
            if dist_p < dist :
                dist = dist_p
                i = p
        centroids[k] = points[i]
        
    return centroids

def find_best_centroids(points):
    best_centroids = []
    best_score = -1.0
    for k in range(points.len()).pop(0):
        current_score = -1
        current_centroids = k_means_plus_plus(points, k)
        current_score = calculate_silhouette_score(points, current_centroids)
        if(current_score >= best_score):
            best_centroids = current_centroids
            best_score = current_score
        else
            return best_centroids

In [ ]:
points_ = []
#initialize vector of points to be used in Kmeans++
for p in range(full_graph.number_of_nodes()):
    points_.append(Point(model.wv[p], -1))

best_c = find_best_centroids(points_)
clustered = []

for p in points_:
    min_dist = float('inf')
    for k in range(best_c.len()):
        distance = distance(p, best_c[k])
        if(distance < min_dist):
            min_dist = distance
            p.assigned = k
            
target_cluster = points_[target].assigned

for p in points_:
    if p.assigned == target_cluster:
        clustered.append(p)

clustered_graph = full_graph.subgraph(clustered)

In [ ]:
#Clustering global cc
average_cluster_coef = nx.average_clustering(clustered_graph)

print(f"Coefficiente di clustering medio: {average_cluster_coef}")



#Clustering per ogni nodo (restituisce un dizionario)
node_cluster_coefs = nx.clustering(clustered_graph)

for node, cluster_coef in node_cluster_coefs.items():
    print(f"Nodo {node}: Coefficiente di clustering = {cluster_coef}")


Valutazione nodi (usare clustered_graph)

In [ ]:
#Valutazione nodi con CC o altre metriche

In [ ]:
#Ricerca cliques

#lettura file di esempio, costruzione nodes e edges
import os
import networkx as nx

current_directory = os.path.dirname(os.path.realpath("__file__"))
filename = "grafo_esempio.txt"    

graph = nx.Graph()
graph = nx.read_edgelist(filename, nodetype=int, create_using=nx.DiGraph)
nx.set_edge_attributes(graph, 1, name='weight')
print(graph)

print("Edges:", graph.edges)
print("Unique Node IDs:", graph.nodes)

In [ ]:
#funzione per il neighbor
def n(v, edges):
    neighbors = set()
    for edge in edges:
        if v in edge:
            neighbors.update(edge)

    neighbors.discard(v)

    return list(neighbors)

#Bron-Kerbosch algorithm with pivot
def BronKerbosch(R, P, X, edges, cliques):
    if not P and not X:
        # P and X are both empty, report R as a maximal clique
        cliques.append(R)
        return

    # Choose a pivot vertex u in P ⋃ X
    pivot = (set(P) | set(X)).pop()

    for v in set(P) - set(n(pivot, edges)):
        # Recursively explore the neighborhood of v
        BronKerbosch(R + [v], list(set(P) & set(n(v, edges))), list(set(X) & set(n(v, edges))), edges, cliques)

        # Remove v from P and add it to X
        P.remove(v)
        X.append(v)

#funzione per la clique massima
def maxClique(cliques):
    return max(cliques, key=len, default=[])

#Ricerca e stampa le cliques e la clique massima
all_cliques = []
node_cliques = []
selected_node = 10
BronKerbosch([], list(graph.nodes), [], graph.edges, all_cliques)
print("Cliques:")
for clique in all_cliques:
    if selected_node in clique:
        node_cliques.append(clique)
        
print(node_cliques)

print("Maximum Clique:", maxClique(node_cliques))

In [ ]:
#Clustering global cc
average_cluster_coef = nx.average_clustering(graph)

print(f"Coefficiente di clustering medio: {average_cluster_coef}")



#Clustering per ogni nodo (restituisce un dizionario)
node_cluster_coefs = nx.clustering(graph)

for node, cluster_coef in node_cluster_coefs.items():
    print(f"Nodo {node}: Coefficiente di clustering = {cluster_coef}")


In [ ]:
#node clique with max cc
max_cc = 0
best_clique = []
for clique in node_cliques:
    avg_cc = 0;
    for node in clique:
        avg_cc += node_cluster_coefs[node]
        
    if avg_cc/len(clique) > max_cc:
        best_clique = clique
        
print(best_clique)

In [ ]:
#Tentativi con rimozioni casuali
import random

# Numero di nodi da rimuovere ad ogni iterazione
n_nodes_to_remove = 1

# Numero totale di iterazioni
n_iterations = 10

# Copia del grafo iniziale
graph_to_modify = full_graph.copy()

for iteration in range(n_iterations):
    # Rimuovi n nodi casuali dal grafo
    nodes_to_remove = random.sample(list(graph_to_modify.nodes()), n_nodes_to_remove)
    updated_graph = graph_to_modify.copy()
    updated_graph.remove_nodes_from(nodes_to_remove)

    # Stampa o elabora i risultati desiderati
    print(f"Iteration {iteration + 1}: Removed nodes {nodes_to_remove}")
    
    # Aggiorna il grafo originale per la prossima iterazione
    original_graph = updated_graph.copy()
